<a href="https://colab.research.google.com/github/eanorambuena/2024_web/blob/main/A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Requirements

Instala la librería que permite calcular el coverage.

In [276]:
from IPython.display import clear_output
%pip install coverage
clear_output()
import unittest

Permite ejecutar los test de una test class particular.

In [277]:
def runTests(testClass):
    loader = unittest.TestLoader()
    suite = loader.loadTestsFromTestCase(testClass)
    runner = unittest.TextTestRunner()
    runner.run(suite)


# Class Under Tests - No modificar

Clase Clock que permite obtener la hora del sistema.

In [278]:
%%writefile clock.py
from datetime import datetime

class Clock:
    count = 0

    def __init__(self):
        Clock.count += 1

    def now(self):
        return datetime.now()

    @classmethod
    def get_instance_count(cls):
        return cls.count

Overwriting clock.py


Clase UserSession

In [279]:
%%writefile user_session.py
from clock import Clock
from datetime import timedelta

class UserSession:
    def __init__(self, user_id: str, clock: Clock = None):
        self.user_id = user_id
        self.logged_in = False
        self.login_time = None
        self.failed_attempts = 0
        self.clock = clock or Clock()

    def login(self, password: str) -> None:
        if self.logged_in:
            raise ValueError("User already logged in")
        if not self._check_password(password):
            self.failed_attempts += 1
            if self.failed_attempts >= 3:
                raise ValueError("Too many failed attempts")
            raise ValueError("Invalid password")
        self.logged_in = True
        self.login_time = self.clock.now()
        self.failed_attempts = 0

    def logout(self) -> None:
        if not self.logged_in:
            raise ValueError("User not logged in")
        self.logged_in = False
        self.login_time = None

    def is_session_active(self) -> bool:
        if not self.logged_in:
            return False
        if self.clock.now() - self.login_time > timedelta(minutes=30):
            self.logged_in = False
            return False
        return True

    def _check_password(self, password: str) -> bool:
        return password == "secure123"

Overwriting user_session.py


# Actividad 1: Crear unit test para verificar la funcionalidad de la clase UserSession.

- El objetivo es tener un coverage de 100%
- Los test deben considerar los principios FIRST. En particular replicable, debemos tener los mismos resultados al ejecutar los test a cualquier hora del dia o en diferente computador.
- Los test no deben depender de otras clases del sistema. Utilice Mocks para no depender de la clase Clock.

In [280]:
%%writefile test_user_session.py

import unittest
from unittest.mock import Mock
from datetime import datetime, timedelta
from user_session import *


class TestUserSession(unittest.TestCase):
    successful_password = "secure123"
    failing_password = Mock()

    def test_successful_login(self):
        user_id_mock = Mock()
        clock_mock = Mock()

        user_session_to_test = UserSession(user_id_mock, clock_mock)
        user_session_to_test.login(self.successful_password)

        self.assertTrue(user_session_to_test.logged_in)

    def test_repeated_successful_login(self):
        user_id_mock = Mock()
        clock_mock = Mock()

        user_session_to_test = UserSession(user_id_mock, clock_mock)
        user_session_to_test.login(self.successful_password)

        self.assertRaisesRegex(ValueError, "User already logged in", user_session_to_test.login, self.successful_password)

    def test_failed_login(self):
        user_id_mock = Mock()
        clock_mock = Mock()

        user_session_to_test = UserSession(user_id_mock, clock_mock)

        self.assertRaisesRegex(ValueError, "Invalid password", user_session_to_test.login, self.failing_password)

    def test_acceptable_repeated_failed_login(self):
        user_id_mock = Mock()
        clock_mock = Mock()

        user_session_to_test = UserSession(user_id_mock, clock_mock)
        try:
            user_session_to_test.login(self.failing_password)
        except:
            pass

        self.assertRaisesRegex(ValueError, "Invalid password", user_session_to_test.login, self.failing_password)

    def test_too_many_repeated_failed_login(self):
        user_id_mock = Mock()
        clock_mock = Mock()

        user_session_to_test = UserSession(user_id_mock, clock_mock)
        for i in range(2):
            try:
                user_session_to_test.login(self.failing_password)
            except:
                pass

        self.assertRaisesRegex(ValueError, "Too many failed attempts", user_session_to_test.login, self.failing_password)

    def test_logged_in_logout(self):
        user_id_mock = Mock()
        clock_mock = Mock()

        user_session_to_test = UserSession(user_id_mock, clock_mock)
        user_session_to_test.login(self.successful_password)

        user_session_to_test.logout()
        self.assertFalse(user_session_to_test.logged_in)

    def test_not_logged_in_logout(self):
        user_id_mock = Mock()
        clock_mock = Mock()

        user_session_to_test = UserSession(user_id_mock, clock_mock)

        self.assertRaisesRegex(ValueError, "User not logged in", user_session_to_test.logout)

    def test_not_logged_in_is_session_active(self):
        user_id_mock = Mock()
        clock_mock = Mock()

        user_session_to_test = UserSession(user_id_mock, clock_mock)

        self.assertFalse(user_session_to_test.is_session_active())

    def test_logged_in_is_session_active(self):
        user_id_mock = Mock()
        clock_mock = Mock()
        clock_mock.now = Mock(return_value=timedelta(0))

        user_session_to_test = UserSession(user_id_mock, clock_mock)
        user_session_to_test.login(self.successful_password)

        self.assertTrue(user_session_to_test.is_session_active())

    def test_logged_in_with_too_much_elapsed_time_is_session_active(self):
        user_id_mock = Mock()
        clock_mock = Mock()
        clock_mock.now = Mock(return_value=timedelta(0))

        user_session_to_test = UserSession(user_id_mock, clock_mock)
        user_session_to_test.login(self.successful_password)
        clock_mock.now = Mock(return_value=timedelta(minutes=30.0001))

        self.assertFalse(user_session_to_test.is_session_active())

    def test_logged_out_is_session_active(self):
        user_id_mock = Mock()
        clock_mock = Mock()

        user_session_to_test = UserSession(user_id_mock, clock_mock)
        user_session_to_test.login(self.successful_password)
        user_session_to_test.logout()

        self.assertFalse(user_session_to_test.is_session_active())


if __name__ == '__main__':
    unittest.main()

Overwriting test_user_session.py


# Meta-Test: Prueba tu solución

**Importante** para ejecutar las siguientes celdas, reinicia sesion (*"Restart Session"*).

Nota que las celdas anteriores exportan su contenido a dos archivos user_session.py y test_user_session.py automaticamente gracias a los comandos *%%writefile*.

En ocaciones si actualizas el contenido de estas celdas estos archivos no se actualizan, por lo que se recomienda "Restart Session" para forsar que estos archivos se actualizen y las celdas se ejecuten con la ultima version de tu codigo en el jupyter book.

## Test que verifica que no se crearon instancias de la clase Clock

In [281]:
from clock import Clock
from test_user_session import TestUserSession
runTests(TestUserSession)
Clock.get_instance_count()

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


0

## Test que mide el coverage.

In [282]:
!coverage run --source=user_session test_user_session.py
!coverage report -m

...........
----------------------------------------------------------------------
Ran 11 tests in 0.010s

OK
Name              Stmts   Miss  Cover   Missing
-----------------------------------------------
user_session.py      34      0   100%
-----------------------------------------------
TOTAL                34      0   100%
